# Preprocess, train and use Trustee

## Imports for pre processing

In [1]:
import pandas as pd
import numpy as np
import pickle
import argparse

## Constants (Directly imported from the puffer paper)

In [2]:
VIDEO_DURATION = 180180
PKT_BYTES = 1500
MILLION = 1000000
PAST_CHUNKS = 8
FUTURE_CHUNKS = 5

## Steps:
    1. 2 CSVs video sent and video acked
    2. Parse them to relevant data types
    3. calculate the transmission time from sent and acked
    4. Consider the past 8 chunks and pad the values if necessary
    5. Create the examples for the future chunks

In [3]:
def prepare_raw_data(video_sent_path, video_acked_path, time_start=None, time_end=None):
    """
    Load data from files and calculate chunk transmission times.
    """
    video_sent_df = pd.read_csv(video_sent_path)
    video_acked_df = pd.read_csv(video_acked_path)

    # Rename "time (ns GMT)" to "time" for convenience
    video_sent_df.rename(columns={'time (ns GMT)': 'time'}, inplace=True)
    video_acked_df.rename(columns={'time (ns GMT)': 'time'}, inplace=True)

    # Convert nanosecond timestamps to datetime
    video_sent_df['time'] = pd.to_datetime(video_sent_df['time'], unit='ns')
    video_acked_df['time'] = pd.to_datetime(video_acked_df['time'], unit='ns')

    # Filter by time range
    if time_start:
        time_start = pd.to_datetime(time_start)
        video_sent_df = video_sent_df[video_sent_df['time'] >= time_start]
        video_acked_df = video_acked_df[video_acked_df['time'] >= time_start]
    if time_end:
        time_end = pd.to_datetime(time_end)
        video_sent_df = video_sent_df[video_sent_df['time'] <= time_end]
        video_acked_df = video_acked_df[video_acked_df['time'] <= time_end]

    # Process the data
    return calculate_trans_times(video_sent_df, video_acked_df)

In [4]:
def calculate_trans_times(video_sent_df, video_acked_df):
    """
    Calculate transmission times from video_sent and video_acked datasets using session_id.
    """
    d = {}
    last_video_ts = {}

    for _, row in video_sent_df.iterrows():
        session = row['session_id']  # Use only session_id to track sessions
        if session not in d:
            d[session] = {}
            last_video_ts[session] = None

        video_ts = int(row['video_ts'])
        if last_video_ts[session] is not None:
            if video_ts != last_video_ts[session] + VIDEO_DURATION:
                continue

        last_video_ts[session] = video_ts
        d[session][video_ts] = {
            'sent_ts': pd.Timestamp(row['time']),
            'size': float(row['size']) / PKT_BYTES,
            'delivery_rate': float(row['delivery_rate']) / PKT_BYTES,
            'cwnd': float(row['cwnd']),
            'in_flight': float(row['in_flight']),
            'min_rtt': float(row['min_rtt']) / MILLION,
            'rtt': float(row['rtt']) / MILLION,
        }

    for _, row in video_acked_df.iterrows():
        session = row['session_id']  # Use only session_id
        if session not in d:
            continue

        video_ts = int(row['video_ts'])
        if video_ts not in d[session]:
            continue

        dsv = d[session][video_ts]
        sent_ts = dsv['sent_ts']
        acked_ts = pd.Timestamp(row['time'])
        dsv['acked_ts'] = acked_ts
        dsv['trans_time'] = (acked_ts - sent_ts).total_seconds()

    return d

In [5]:
def append_past_chunks(ds, next_ts, row):
    i = 1
    past_chunks = []
    while i <= PAST_CHUNKS:
        ts = next_ts - i * VIDEO_DURATION
        if ts in ds and 'trans_time' in ds[ts]:
            past_chunks = [ds[ts]['delivery_rate'],
                           ds[ts]['cwnd'], ds[ts]['in_flight'],
                           ds[ts]['min_rtt'], ds[ts]['rtt'],
                           ds[ts]['size'], ds[ts]['trans_time']] + past_chunks
        else:
            nts = ts + VIDEO_DURATION  # padding with the nearest ts
            padding = [ds[nts]['delivery_rate'],
                       ds[nts]['cwnd'], ds[nts]['in_flight'],
                       ds[nts]['min_rtt'], ds[nts]['rtt']]
            if nts == next_ts:
                padding += [0, 0]  # next_ts is the first chunk to send
            else:
                padding += [ds[nts]['size'], ds[nts]['trans_time']]
            break
        i += 1
    if i != PAST_CHUNKS + 1:  # break in the middle; padding must exist
        while i <= PAST_CHUNKS:
            past_chunks = padding + past_chunks
            i += 1
    row += past_chunks

In [14]:
def prepare_input_output(d):
    ret = [{'in': [], 'out': []} for _ in range(5)]  # FUTURE_CHUNKS = 5

    for session in d:
        ds = d[session]

        for next_ts in ds:
            if 'trans_time' not in ds[next_ts]:
                continue

            row = []

            # Append past chunks
            append_past_chunks(ds, next_ts, row)

            # Append the TCP info of the next chunk
            row += [ds[next_ts]['delivery_rate'],
                    ds[next_ts]['cwnd'], ds[next_ts]['in_flight'],
                    ds[next_ts]['min_rtt'], ds[next_ts]['rtt']]

            # Generate FUTURE_CHUNKS rows
            for i in range(5):  # FUTURE_CHUNKS = 5
                row_i = row.copy()

                ts = next_ts + i * VIDEO_DURATION
                if ts in ds and 'trans_time' in ds[ts]:
                    row_i += [ds[ts]['size']]

                    ret[i]['in'].append(row_i)
                    ret[i]['out'].append(ds[ts]['trans_time'])

    return ret

In [6]:
def save_processed_data(output_file, processed_data):
    """
    Save processed data to a file.
    """
    with open(output_file, 'wb') as f:
        pickle.dump(processed_data, f)
    print(f"Processed data saved to {output_file}")

In [ ]:
if __name__ == '__main__':
    DEFAULT_VIDEO_SENT_PATH = '/mnt/md0/jaber/puffer_train/video_sent_2022-10-01T11_2022-10-02T11.csv'
    DEFAULT_VIDEO_ACKED_PATH = '/mnt/md0/jaber/puffer_train/video_acked_2022-10-01T11_2022-10-02T11.csv'
    DEFAULT_OUTPUT_FILE = '/mnt/md0/<user_id>/old_output.pkl'
    
    #Latest datasets can be found at https://puffer.stanford.edu/results/
    
    parser = argparse.ArgumentParser(description="Process video streaming datasets.")
    parser.add_argument('--video_sent_path', type=str, help='Path to the video_sent dataset CSV file.')
    parser.add_argument('--video_acked_path', type=str, help='Path to the video_acked dataset CSV file.')
    parser.add_argument('--output_file', type=str, help='Path to save the processed data.')
    parser.add_argument('--time_start', type=str, default=None, help='Start time for filtering data (RFC3339 format).')
    parser.add_argument('--time_end', type=str, default=None, help='End time for filtering data (RFC3339 format).')
    #args = parser.parse_args()
    #processed_data = prepare_input_output(prepare_raw_data(args.video_sent_path, args.video_acked_path,
    #    time_start=args.time_start, time_end=args.time_end))
    # save_processed_data(args.output_file, processed_data)
    processed_data = prepare_input_output(prepare_raw_data(DEFAULT_VIDEO_SENT_PATH, DEFAULT_VIDEO_ACKED_PATH,
        time_start=None, time_end=None))
    save_processed_data(DEFAULT_OUTPUT_FILE, processed_data)

## Model selection
1. Define the model.
2. Normalize the data
3. Discretize the labels
4. Plot the classification report
5. Use trustee to identify the important features
6. Plot the Trustee tree

In [ ]:
#model side imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import pickle
import numpy as np
from os import path
from sklearn.metrics import mean_squared_error
import pandas as pd
from trustee import ClassificationTrustee
import graphviz
from sklearn import tree

In [ ]:
# Constants
BATCH_SIZE = 32
NUM_EPOCHS = 500
DEVICE = torch.device('cpu')

inference = True

In [ ]:
class Model:
    #Model constants
    PAST_CHUNKS = 8
    FUTURE_CHUNKS = 5
    DIM_IN = 62
    COLUMNS = [j + str(i) for i in range(PAST_CHUNKS + 1) for j in ['delivery_rate', 'cwnd', 'in_flight', 'min_rtt', 'rtt', 'size', 'trans_time']][:DIM_IN]
    DIM_OUT = 21  # BIN_MAX + 1
    DIM_H1 = 64
    DIM_H2 = 64
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-4
    BIN_SIZE = 0.5  # seconds
    BIN_MAX = 20

    def __init__(self):
        self.model = torch.nn.Sequential(
            torch.nn.Linear(Model.DIM_IN, Model.DIM_H1),
            torch.nn.ReLU(),
            torch.nn.Linear(Model.DIM_H1, Model.DIM_H2),
            torch.nn.ReLU(),
            torch.nn.Linear(Model.DIM_H2, Model.DIM_OUT),
        ).double().to(device=DEVICE)
        self.loss_fn = torch.nn.CrossEntropyLoss().to(device=DEVICE)
        self.optimizer = torch.optim.Adam(self.model.parameters(),
                                          lr=Model.LEARNING_RATE,
                                          weight_decay=Model.WEIGHT_DECAY)
        self.obs_size = None
        self.obs_mean = None
        self.obs_std = None

    def update_obs_stats(self, raw_in):
        if self.obs_size is None:
            self.obs_size = len(raw_in)
            self.obs_mean = np.mean(raw_in, axis=0)
            self.obs_std = np.std(raw_in, axis=0)
            return
        old_size = self.obs_size
        new_size = len(raw_in)
        self.obs_size = old_size + new_size
        old_mean = self.obs_mean
        new_mean = np.mean(raw_in, axis=0)
        self.obs_mean = (old_mean * old_size + new_mean * new_size) / self.obs_size
        old_std = self.obs_std
        old_sum_square = old_size * (np.square(old_std) + np.square(old_mean))
        new_sum_square = np.sum(np.square(raw_in), axis=0)
        mean_square = (old_sum_square + new_sum_square) / self.obs_size
        self.obs_std = np.sqrt(mean_square - np.square(self.obs_mean))

    def normalize_input(self, raw_in, update_obs=False):
        z = np.array(raw_in)
        if update_obs:
            self.update_obs_stats(z)
        assert self.obs_size is not None
        for col in range(len(self.obs_mean)):
            z[:, col] -= self.obs_mean[col]
            if self.obs_std[col] != 0:
                z[:, col] /= self.obs_std[col]
        return z

    def discretize_output(self, raw_out):
        z = np.array(raw_out)
        z = np.floor((z + 0.5 * Model.BIN_SIZE) / Model.BIN_SIZE).astype(int)
        return np.clip(z, 0, Model.BIN_MAX)

    def train(self, train_input, train_output, test_input, test_output):
        train_input = torch.from_numpy(self.normalize_input(train_input, update_obs=inference)).to(DEVICE)
        train_output = torch.from_numpy(self.discretize_output(train_output)).to(DEVICE)
        test_input = torch.from_numpy(self.normalize_input(test_input, update_obs=False)).to(DEVICE)
        test_output = torch.from_numpy(self.discretize_output(test_output)).to(DEVICE)

        for epoch in range(NUM_EPOCHS):
            self.model.train()
            perm = np.random.permutation(len(train_input))
            train_input = train_input[perm]
            train_output = train_output[perm]

            num_batches = int(np.ceil(len(train_input) / BATCH_SIZE))
            epoch_loss = 0

            for i in range(num_batches):
                start_idx = i * BATCH_SIZE
                end_idx = min((i + 1) * BATCH_SIZE, len(train_input))

                batch_input = train_input[start_idx:end_idx]
                batch_output = train_output[start_idx:end_idx]

                # Forward pass
                predictions = self.model(batch_input)
                loss = self.loss_fn(predictions, batch_output)

                # Backward pass
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

            print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {epoch_loss / num_batches}")

            # Evaluate after each epoch
            self.evaluate(test_input, test_output)

    def load(self, model_path):
        checkpoint = torch.load(model_path)
        self.model.load_state_dict(checkpoint['model_state_dict'])

        self.obs_size = checkpoint['obs_size']
        self.obs_mean = checkpoint['obs_mean']
        self.obs_std = checkpoint['obs_std']

    def save(self, model_path):
        assert (self.obs_size is not None)

        torch.save({
            'model_state_dict': self.model.state_dict(),
            'obs_size': self.obs_size,
            'obs_mean': self.obs_mean,
            'obs_std': self.obs_std,
        }, model_path)

    def predict(self, x):
        with torch.no_grad():
            x = x.to_numpy()
            x = self.normalize_input(x, update_obs=inference)
            x = torch.from_numpy(x).to(DEVICE)
            y_scores = self.model(x)
            y_predicted = torch.max(y_scores, 1)[1].to(device=DEVICE)
            ret = y_predicted.detach().cpu().numpy()
            return ret
    def predict_discrete(self, x):
        with torch.no_grad():
            x = x.to_numpy()
            x = self.normalize_input(x, update_obs=inference)
            x = torch.from_numpy(x).to(DEVICE)
            y_scores = self.model(x)
            y_predicted = torch.max(y_scores, 1)[1].to(device=DEVICE)
            ret = y_predicted.detach().cpu().numpy()
            return y_scores, ret

    def predict_cont(self, x):
        with torch.no_grad():
            x = x.to_numpy()
            x = self.normalize_input(x, update_obs=inference)
            x = torch.from_numpy(x).to(DEVICE)
            y_scores = self.model(x)
            y_predicted = torch.max(y_scores, 1)[1].to(device=DEVICE)
            ret = y_predicted.double().numpy()
            for i in range(len(ret)):
                bin_id = ret[i]
                if bin_id == 0:  # the first bin is defined differently
                    ret[i] = 0.25 * Model.BIN_SIZE
                else:
                    ret[i] = bin_id * Model.BIN_SIZE
            return ret

    def evaluate_with_trustee(self, test_input, test_output):
        self.model.eval()
        pd_input = pd.DataFrame(test_input, columns=self.COLUMNS)
        test_output_discretized = self.discretize_output(test_output)
        with torch.no_grad():
            predictions_prob, class_preds = self.predict_discrete(pd_input)
            # Cross-Entropy Loss
            cross_entropy_loss = self.loss_fn(predictions_prob, torch.from_numpy(test_output_discretized)).item()
            # Print metrics
            print(f"Test Cross-Entropy Loss: {cross_entropy_loss}")
            print("Classification Report:")
            print(classification_report(test_output_discretized, class_preds, zero_division=0))
            trustee = ClassificationTrustee(expert=model)
            trustee.fit(pd_input, test_output_discretized, num_iter=10, num_stability_iter=2, samples_size=0.3, verbose=True)
            dt, pruned_dt, agreement, reward = trustee.explain()
            dt_y_pred = dt.predict(pd_input)
            print("Model explanation global fidelity report:")
            print(classification_report(class_preds, dt_y_pred))
            dot_data = tree.export_graphviz(pruned_dt, class_names=[str(i)for i in range(21)], feature_names=model.COLUMNS,filled=True,rounded=True,special_characters=True)
            graph = graphviz.Source(dot_data)
            fil = graph.render("~/trustee_tree_puffer_pruned", format="png")

    def evaluate(self, test_input, test_output):
        self.model.eval()
        pd_input = pd.DataFrame(test_input, columns=self.COLUMNS)
        with torch.no_grad():
            predictions = self.predict_cont(pd_input)
            # Print metrics

            # Mean Squared Error
            mse_loss = mean_squared_error(test_output, predictions)
            print(f"Test Mean Squared Error: {mse_loss}")


In [ ]:
with open('/mnt/md0/satya/old_output.pkl', 'rb') as f:
    processed_data = pickle.load(f)
model = Model()

for i, chunk_data in enumerate(processed_data):
    input_data = np.array(chunk_data['in'])
    output_data = np.array(chunk_data['out'])
    if not inference:
        print(f"Training model for future chunk {i + 1}")
        # Train-test split (70% train, 30% test)
        train_input, test_input, train_output, test_output = train_test_split(
            input_data, output_data, test_size=0.3, random_state=42
        )
        model.train(train_input, train_output, test_input, test_output)
    else:
        model.load("/mnt/md0/satya/puffer/src/model/py-0-checkpoint-200.pt")
        # the model is available at https://storage.googleapis.com/puffer-models/puffer-ttp/bbr-20221001-1.tar.gz
        model.evaluate(input_data, output_data)

In [ ]:
model.evaluate_with_trustee(input_data, output_data)

![title](trustee_tree_puffer_pruned.png)